In [1]:
import random
import operator
import matplotlib.pyplot as pl
import math

In [31]:
def number_of_colors(G):   #número de cores que será utilizado para colorir o grafo, definimos como 2 vezes o grau máximo
    max_degree = max(G.degree())
    return max_degree + 1



def energy(G):  #calcula o número de colisões, ou seja, a quantidade de vezes em que duas arestas
                #distintas adjacentes foram coliridas com a mesma cor
    
    ones = 0
    neighbors = []
    edges_seen = []

    for edge in G.edges():
        neighbors = []
        v1, v2 = edge[0] , edge[1]
        if (v1,v2) not in edges_seen and (v2,v1) not in edges_seen:
            for element in G.edges_incident(v1):
                if element[1] != v2:
                    neighbors.append(element)
            for element in G.edges_incident(v2):
                if element[0] != v1:
                    neighbors.append(element)
            for element in neighbors:
                if edge[2] == element[2]:
                    ones += 1
            edges_seen.append((v1,v2))
        #print(neighbors, edge[0], edge[1])
    return ones / 2
        
        
def random_color(G):  #colori de forma aleatória as arestas do grafo
    k = number_of_colors(G) 
    for element in G.edges():
        choice = random.randint(0,number_of_colors(G)-1)
        G.set_edge_label(element[0],element[1],choice)
    
    
def transition(edge_atual,current): #escolhe uma aresta adjacente para transitar
    edges_seen = []
    neighbors = []
    v1, v2 = edge_atual[0] , edge_atual[1]
    if (v1,v2) not in edges_seen and (v2,v1) not in edges_seen:
            for element in current.edges_incident(v1):
                if element[1] != v2:
                    neighbors.append(element)
            for element in current.edges_incident(v2):
                if element[0] != v1:
                    neighbors.append(element)
    choice = random.choice(neighbors) #escolhe um vizinho aleatório

    return choice


def acceptance_probability(curr_energy, candidate_energy,temp):
    '''
    Acceptance probability using boltzmann:
    '''
    
    return math.exp(-abs(candidate_energy - curr_energy) / temp)


def annealing(G, temp = 100, stopping_iteration = 100, alpha = 0.9):
    random_color(G)
    colors = number_of_colors(G)
    i = 100
    initial_energy = energy(G)
    print('energia inicial {}'.format(initial_energy))
    current = G.copy()
    current_energy = initial_energy
    current_edge = random.choice(G.edges())
    #print('aresta inicial {}'.format(current_edge))
    best_solution = Graph()
    G.show(color_by_label = True)

    
    stopping_temp = 0.0001
    iteration = 0
    alpha = 0.7
    
    best_solution_energy = 10000
    
    while temp >= stopping_temp: 
        while iteration < stopping_iteration:
    
            #transição
            new_edge = transition(current_edge,current)
            #print('nova aresta {}'.format(new_edge))
            new_color = random.randint(0,colors - 1)  #Atribuí uma cor ao vizinho de forma aleatória
            #print('nova cor {}'.format(new_color))
            #current.show(color_by_label=True)
            candidate = current.copy() #cópia do grafo atual
            candidate.set_edge_label(new_edge[0],new_edge[1],new_color) #modifica a cor da aresta, transitando. 
            candidate_energy = energy(candidate)
            #print('energia candidato {}'.format(candidate_energy))
            #candidate.show(color_by_label=True)

            #aceitação

            if candidate_energy <= current_energy: #aceitamos e atualizamos o grafo atual
                if candidate_energy == 0:
                    best_solution = candidate.copy()
                    best_solution_energy = energy(best_solution)
                    break
                current = candidate.copy()
                current_energy = candidate_energy
                current_edge = new_edge
                #print('A energia era menor, logo aceitamos')
            else:
                unif = random.random() # generate a uniform number between 0 and 1
                if unif < acceptance_probability(current_energy, candidate_energy,temp):
                    if candidate_energy == 0:
                        best_solution = candidate.copy()
                        best_solution_energy = energy(best_solution)
                        break

                    current = candidate.copy()
                    current_energy = candidate_energy
                    current_edge = new_edge
                    #print('aceitou por boltzmann')
            
            iteration += 1

        if best_solution_energy == 0:
            best_solution.show(color_by_label = True)
            return best_solution_energy
        
        #resfriamento

        temp = temp * alpha
        iteration = 0




In [ ]:
teste = graphs.RandomGNP(20,1)
print(teste.is_connected())
teste.show(color_by_label=True)

In [ ]:
k = 1

while k != 0:
    k = annealing(teste,temp = 1000, stopping_iteration = 10000, alpha = 0.95)